In [61]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/Users/macbookpro/Desktop/lxc/git/Crash-Severity-Prediction/Crash_Reporting_-_Drivers_Data.csv")

# Step 1: data cleaning 
# Check for missing values
"""
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)
"""
# drop large data missing column
columns_to_drop = ['Off-Road Description', 'Municipality', 'Related Non-Motorist', 
                   'Non-Motorist Substance Abuse', 'Circumstance', 'Equipment Problems']

df1 = df.drop(columns=columns_to_drop)
"""
# missing_values_update = df1.isnull().sum()
# print("Missing Values:")
# print(missing_values_update)
"""
### Since there are over 160,000 data, we can drop unuse data. 
df2 = df1.dropna()
"""
# missing_values_update1 = df2.isnull().sum()
# print("Missing Values:")
# print(missing_values_update1)
"""

### Drop irrevelant columns
col= ['Report Number',"Road Name", 'Local Case Number', 'Agency Name', 'Person ID', 'Vehicle ID']
df3 = df2.drop(columns=col)
print("DataFrame size (rows, columns):", df3.shape)

# Check the number of rows containing "Unknown" in the entire DataFrame
unknown_rows_count = (df3 == 'Unknown').sum().sum()
other = (df3 == 'OTHER').sum().sum()
UNKNOWN= (df3 == 'UNKNOWN').sum().sum()


# Drop rows with "Unknown" in the entire DataFrame
df4 = df3[df3 != 'Unknown'].dropna()
df4 = df4[df4 != 'OTHER'].dropna()
df4 = df4[df4 != 'UNKNOWN'].dropna()

# Check the DataFrame size after dropping "Unknown" rows
print("Number of rows with 'Unknown' before dropping:", unknown_rows_count)
print("DataFrame size after dropping 'Unknown' rows (rows, columns):", df4.shape)

#check the data type for each column#
"""
data_types = df4.dtypes
print(data_types)
"""

# Step 2: Encoding Categorical Variables
# Get the list of categorical columns (assuming you have already dropped the unnecessary columns)
categorical_columns = df4.select_dtypes(include=['object']).columns

# Loop through each categorical column and count the unique categories
for column in categorical_columns:
    unique_categories = df4[column].nunique()
    # print(f"Column: {column}, Unique Categories: {unique_categories}")

# Step 3: Handling Date/Time Columns
df4['Crash Date/Time'] = pd.to_datetime(df4['Crash Date/Time'])
df4['Hour'] = df4['Crash Date/Time'].dt.hour
df4['Day'] = df4['Crash Date/Time'].dt.day
df4['Month'] = df4['Crash Date/Time'].dt.month
df4['Year'] = df4['Crash Date/Time'].dt.year

# Drop unusable column:
col2= ['Crash Date/Time','Vehicle Make','Cross-Street Name','ACRS Report Type','Cross-Street Type','Vehicle Model',"Parked Vehicle",'Driverless Vehicle']
df5 = df4.drop(columns=col2)
print("DataFrame size (rows, columns):", df5.shape)

categorical_columns = df5.select_dtypes(include=['object']).columns
# Loop through each categorical column and count the unique categories
for column in categorical_columns:
    unique_categories = df5[column].nunique()
    print(f"Column: {column}, Unique Categories: {unique_categories}")



/var/folders/9z/gkk8fkkj0pz746hrzcw69dq40000gn/T/ipykernel_12823/1098812055.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/macbookpro/Desktop/lxc/git/Crash-Severity-Prediction/Crash_Reporting_-_Drivers_Data.csv")


DataFrame size (rows, columns): (95459, 31)
Number of rows with 'Unknown' before dropping: 10808
DataFrame size after dropping 'Unknown' rows (rows, columns): (66640, 31)
DataFrame size (rows, columns): (66640, 27)
Column: Route Type, Unique Categories: 9
Column: Collision Type, Unique Categories: 16
Column: Weather, Unique Categories: 10
Column: Surface Condition, Unique Categories: 9
Column: Light, Unique Categories: 6
Column: Traffic Control, Unique Categories: 9
Column: Driver Substance Abuse, Unique Categories: 9
Column: Driver At Fault, Unique Categories: 2
Column: Injury Severity, Unique Categories: 5
Column: Driver Distracted By, Unique Categories: 18
Column: Drivers License State, Unique Categories: 68
Column: Vehicle Damage Extent, Unique Categories: 5
Column: Vehicle First Impact Location, Unique Categories: 15
Column: Vehicle Second Impact Location, Unique Categories: 15
Column: Vehicle Body Type, Unique Categories: 28
Column: Vehicle Movement, Unique Categories: 19
Column:

In [62]:
# cate the hour
import numpy as np
# Define a dictionary to map months to seasons
seasons_mapping = {
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall',
    12: 'Winter', 1: 'Winter', 2: 'Winter'
}

# Create a new column 'Season' based on the 'Month' column
df5['Season'] = df5['Month'].map(seasons_mapping)

# Create a new column 'Time of Day' based on the 'Hour' column
conditions = [
    (df5['Hour'] >= 6) & (df5['Hour'] < 12),
    (df5['Hour'] >= 12) & (df5['Hour'] < 18),
    (df5['Hour'] >= 18) & (df5['Hour'] < 24),
    (df5['Hour'] >= 0) & (df5['Hour'] < 6)
]
choices = ['6am-12pm', '12pm-18pm', '18pm-24am', '0am-6am']
df5['Time of Day'] = np.select(conditions, choices)

# Create a new column 'Month Segment' based on the 'Day' column
conditions = [
    (df5['Day'] >= 1) & (df5['Day'] <= 10),
    (df5['Day'] >= 11) & (df5['Day'] <= 20),
    (df5['Day'] >= 21) & (df5['Day'] <= 31)
]
choices = ['Beginning of Month', 'Mid Month', 'End of Month']
df5['Month Segment'] = np.select(conditions, choices)



In [63]:
### THIS IS THE FINAL DATASET WE GOING TO WORK ON ###
col3= ['Hour','Day','Month']
df5 = df5.drop(columns=col3) 

col4= ['Latitude','Longitude','Location']
crash_no_location = df5.drop(columns=col4) 


In [56]:
crash_no_location.head()

,Route Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,Injury Severity,Driver Distracted By,...,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Vehicle Year,Year,Season,Time of Day,Month Segment
2,Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,NO APPARENT INJURY,NOT DISTRACTED,...,PICKUP TRUCK,ACCELERATING,North,North,35,2019,2023,Summer,12pm-18pm,Mid Month
3,County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,Yes,NO APPARENT INJURY,LOOKED BUT DID NOT SEE,...,PASSENGER CAR,STARTING FROM LANE,East,East,40,2016,2023,Summer,12pm-18pm,End of Month
4,County,HEAD ON LEFT TURN,CLOUDY,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,NO APPARENT INJURY,NOT DISTRACTED,...,(SPORT) UTILITY VEHICLE,MAKING LEFT TURN,West,North,35,2016,2023,Summer,6am-12pm,End of Month
5,County,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,No,NO APPARENT INJURY,NOT DISTRACTED,...,TRANSIT BUS,STOPPED IN TRAFFIC LANE,East,East,30,2016,2023,Summer,6am-12pm,Mid Month
6,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DUSK,NO CONTROLS,NONE DETECTED,Yes,NO APPARENT INJURY,NOT DISTRACTED,...,(SPORT) UTILITY VEHICLE,SLOWING OR STOPPING,South,South,25,2014,2023,Summer,18pm-24am,Mid Month


In [64]:
data_types = crash_no_location.dtypes
print("Data Types for Each Column:")
print(data_types)

# Display unique values for each column
for column in crash_no_location.columns:
    unique_values = crash_no_location[column].unique()
    print(f"Unique values for '{column}':")
    print(unique_values)

Data Types for Each Column:
Route Type                        object
Collision Type                    object
Weather                           object
Surface Condition                 object
Light                             object
Traffic Control                   object
Driver Substance Abuse            object
Driver At Fault                   object
Injury Severity                   object
Driver Distracted By              object
Drivers License State             object
Vehicle Damage Extent             object
Vehicle First Impact Location     object
Vehicle Second Impact Location    object
Vehicle Body Type                 object
Vehicle Movement                  object
Vehicle Continuing Dir            object
Vehicle Going Dir                 object
Speed Limit                        int64
Vehicle Year                       int64
Year                               int64
Season                            object
Time of Day                       object
Month Segment                

In [ ]:
# If your dataset contains categorical variables, you'll need to encode them to numerical values for machine learning.
# There are different encoding techniques, but a common one is one-hot encoding.
# For example, if you have a 'Weather' column, you can use pd.get_dummies to create one-hot encoded columns.

# df = pd.get_dummies(df, columns=['Weather', 'Surface Condition', 'Light', 'Driver Substance Abuse', ...])


# Step 5: Scaling/Normalization 

# Depending on the machine learning algorithms you plan to use, you may need to scale or normalize numeric features.
# Popular techniques include Min-Max scaling or Standardization (z-score scaling).

# Standardization example:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df[['NumericFeature1', 'NumericFeature2']] = scaler.fit_transform(df[['NumericFeature1', 'NumericFeature2']])

# Step 6: Data Splitting (for modeling)

# If you plan to build a predictive model, you'll need to split your dataset into training and testing sets.
# This is typically done using the train_test_split function from scikit-learn.

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, you can proceed with EDA and modeling with your preprocessed data.